In [2]:
from collections import deque 

In [14]:
dp = deque(maxlen= 3)



In [19]:
dp.append(1)
dp.append(2)
dp.append(3)


In [ ]:
BASE_DIR = Path(__file__).resolve().parent.parent
DEFAULT_WEIGHTS_PATH = BASE_DIR / "services" / "gelan_t.pt"
from collections import deque
# Lớp Buffer khung ảnh với khả năng quản lý hiệu quả hơn
class FrameBuffer:
    def __init__(self, max_size=5):
        self.input_frames = deque(maxlen=max_size)
        self.processed_frames = queue.Queue()
        self.latest_result = None
        self.processing = True
        self.lock = threading.Lock()

    def put_input(self, frame):
        self.input_frames.append(frame)

    def get_input(self):
        try:
            return self.input_frames.pop()
        except IndexError:
            return None
    def put_processed(self , frame , detection):
        with self.lock:
            self.processed_frames.put((frame , detection))
    def put_result(self, frame, detection):
        with self.lock:
            self.latest_result = (frame, detection)
    def get_processed_frame(self):
        with self.lock:
            return self.processed_frames
    def get_latest(self):
        with self.lock:
            return self.latest_result

    def stop(self):
        self.processing = False


# Hàm YOLO worker (xử lý ảnh)
def yolo_worker(model, frame_buffer, conf_threshold=0.5):
    while frame_buffer.processing:
        frame = frame_buffer.get_input()
        if frame is None:
            time.sleep(0.01)  # Ngủ một chút để tránh tiêu tốn CPU
            continue

        with torch.no_grad():
            # Thực hiện dự đoán YOLO trên frame
            predictions = model(frame)
            # frame_buffer.put_result(frame, predictions)
            frame_buffer.put_processed(frame , predictions)
from queue import Empty
def show_video_stream(video_path, model, device="cpu"):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video file: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_time = 1.0 / fps if fps > 0 else 0.03

    # Khởi tạo buffer và worker threads
    frame_buffer = FrameBuffer(max_size=5)

    # Khởi động 1 worker thread (có thể tăng lên nếu GPU hỗ trợ)
    worker_thread = threading.Thread(
        target=yolo_worker,
        args=(model, frame_buffer, 0.8),
        daemon=True
    )
    worker_thread.start()

    def generate():
        frame_count = 0
        last_time = time.time()

        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_count += 1
                current_time = time.time()
                elapsed = current_time - last_time

                # Đưa frame vào để xử lý
                frame_buffer.put_input(frame.copy())

                # Lấy kết quả mới nhất đã xử lý
                try :
                    result = frame_buffer.get_processed_frame().get_nowait()
                except Empty:
                    result = None
                if result:
                    processed_frame, predictions = result

                    # Vẽ các hộp giới hạn và nhãn
                    if len(predictions.pred) > 0:
                        for det in predictions.pred[0]:
                            x1, y1, x2, y2, conf, class_id = det[:6]
                            if conf < 0.8:
                                continue

                            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                            label = "violence"
                            conf_str = f"{conf:.2f}"

                            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4)
                            cv2.putText(frame, label, (x1, y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                            cv2.putText(frame, conf_str, (x1, y1 - 25),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Hiển thị FPS
                fps_text = f"FPS: {1.0 / elapsed:.1f}" if elapsed > 0 else "FPS: -"
                cv2.putText(frame, fps_text, (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Cập nhật thời gian cho frame tiếp theo
                last_time = current_time

                # Encode frame thành JPEG với chất lượng tốt hơn
                encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 85]
                ret, buffer = cv2.imencode('.jpg', frame, encode_param)
                frame_bytes = buffer.tobytes()

                # Gửi frame đến client
                yield (b'--frame\r\n'
                       b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

                # Kiểm soát frame rate để tránh quá tải CPU
                processing_time = time.time() - current_time
                sleep_time = max(0, frame_time - processing_time)
                if sleep_time > 0:
                    time.sleep(sleep_time)

        finally:
            # Đảm bảo dọn dẹp tài nguyên
            frame_buffer.stop()
            cap.release()

    return StreamingResponse(
        generate(),
        media_type='multipart/x-mixed-replace; boundary=frame'
    )


async def track_video_service(video_id, weights_path=DEFAULT_WEIGHTS_PATH, device=None):
    # Xác định thiết bị phù hợp
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Tìm video trong database
    # from database import video_collection  # Giả sử có import này
    video = await video_collection.find_one({"_id": ObjectId(video_id)})

    if not video:
        raise HTTPException(status_code=404, detail="Video not found")

    video_path = video["file_path"]

    # Kiểm tra xem file video có tồn tại
    if not Path(video_path).exists():
        raise HTTPException(status_code=404, detail="Video file not found on server")

    # Load model YOLO
    try:
        model_backend = DetectMultiBackend(weights=str(weights_path), device=device)
        model = AutoShape(model_backend)
        model.eval()
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to load model: {str(e)}")

    # Trả về stream video
    return show_video_stream(video_path, model, device=device)

In [21]:
a=dp.pop()
a

3